In [2]:
import pandas as pd
from sklearn.model_selection import \
    TimeSeriesSplit  # Découper une série temporelle en prenant en compte la dépendance temporelle
from sklearn.feature_selection import \
    SequentialFeatureSelector  # Sélectionner de manière séquentielle un sous ensemble de caractéristiques à partie d'un ensemble plus large
from sklearn.linear_model import \
    RidgeClassifier  # Fonction linéaire régularisée, prédire la classe d'un échantillon à partir de caractéristiques
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

###################################### TRAITEMENT DE DONNEES ##########################################

df = pd.read_csv("nba_games.csv", index_col=0)

# → Trier les lignes avec la date et obtenir les nouveaux index ensuite, sans créer une colonne des anciens index
df = df.sort_values("date")
df = df.reset_index(drop=True)

del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

# team = box scores for one team
# add_target => inclue une colonne indiquant le résultat de l'équipe au match d'après
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team


df = df.groupby("team", group_keys=False).apply(add_target)
df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

team_df = df[df["team"] == "WAS"]

# Checker toutes les valeurs nulles de notre dataframe : false si c'est not null, true si c'est null
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]
# Sélection des colonnes qui ne sont pas dans nulls grâce à l'opérateur négatif ~
valid_columns = df.columns[~df.columns.isin(nulls.index)]
# Permet de copier les nouvelles colonnes, les ajouter dans des colonnes existantes peuvent créer des erreurs
df = df[valid_columns].copy()

/var/folders/ll/_5vx1kfs0zb1d286dqr8f1zc0000gn/T/ipykernel_43491/2545537586.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


In [3]:
print(df)

         mp    fg   fga    fg%    3p   3pa    3p%    ft   fta    ft%  ...  \
0     240.0  40.0  80.0  0.500  13.0  34.0  0.382  24.0  28.0  0.857  ...   
1     240.0  45.0  99.0  0.455  16.0  45.0  0.356  17.0  23.0  0.739  ...   
2     240.0  40.0  94.0  0.426  10.0  40.0  0.250  19.0  25.0  0.760  ...   
3     240.0  46.0  82.0  0.561  12.0  35.0  0.343  22.0  28.0  0.786  ...   
4     240.0  36.0  86.0  0.419  13.0  30.0  0.433  23.0  32.0  0.719  ...   
...     ...   ...   ...    ...   ...   ...    ...   ...   ...    ...  ...   
1903  240.0  38.0  78.0  0.487   9.0  29.0  0.310  18.0  23.0  0.783  ...   
1904  240.0  36.0  86.0  0.419  10.0  31.0  0.323  20.0  24.0  0.833  ...   
1905  240.0  42.0  89.0  0.472  14.0  34.0  0.412  12.0  16.0  0.750  ...   
1906  240.0  42.0  84.0  0.500  20.0  47.0  0.426  21.0  25.0  0.840  ...   
1907  240.0  46.0  86.0  0.535  10.0  26.0  0.385  15.0  19.0  0.789  ...   

      tov%_max_opp  usg%_max_opp  ortg_max_opp  drtg_max_opp  team_opp  \
0

In [6]:
##################################################### MACHINE LEARNING #################################################################

# Traiter un grand nombre de colonnes en machine learning peut entraîner des erreurs de corrélations, utilisation de sélecteurs
# Machine learning model
rr = RidgeClassifier(alpha=1)  # Alpha pour la régularisation
split = TimeSeriesSplit(n_splits=3)

# Le sélecteur va entraîner la machine en envoyant des parties de features et récupérer les features les plus pertinentes
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

selected_columns = df.columns[~df.columns.isin(removed_columns)]
# Valeurs des colonnes sélectionnées sont entre 0 et 1
scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

# Récupérer les 30 meilleures features pour prédire
sfs.fit(df[selected_columns], df["target"])

# sfs.get_support => True: colonnes pertinentes pour notre prédiction
predictors = list(selected_columns[sfs.get_support()])

In [7]:
def backtest(data, model, predictors, date):
    all_predictions = []

    train = data[data["date"] < date]
    model.fit(train[predictors], train["target"])
    test = data[data["date"] == date]

    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)

    combined = pd.concat([test["target"], preds], axis=1)
    combined.columns = ["actual", "prediction"]

    all_predictions.append(combined)
    return pd.concat(all_predictions)

# Récupérer les colonnes sélectionnées et ajouter les colonnes won et team
df_rolling = df[list(selected_columns) + ["won", "team"]]

# Groupe les 10 dernières lignes d'une équipe de notre df, pour chacune des lignes, retourne la moyenne avec mean
def find_team_averages(team):
    rolling = team.rolling(30).mean()
    return rolling
# Regrouper nos colonnes pour une équipe spécifique
df_rolling = df_rolling.groupby(["team"], group_keys = False).apply(find_team_averages)

rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)
df = df.dropna()

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")
df = df.copy()

full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]],
                left_on=["team", "date_next"],
                right_on=["team_opp_next", "date_next"]
)

/var/folders/ll/_5vx1kfs0zb1d286dqr8f1zc0000gn/T/ipykernel_43491/3336873986.py:22: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['team'], dtype='object')
  rolling = team.rolling(30).mean()


In [8]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns
selected_columns = full.columns[~full.columns.isin(removed_columns)]

sfs.fit(full[selected_columns], full["target"])
predictions = backtest(full, rr, predictors, "2023-03-01")
accuracy = accuracy_score(predictions["actual"], predictions["prediction"])
full = pd.concat([full, predictions], axis=1)

In [9]:
print(full)

      mp        fg       fga       fg%    3p       3pa       3p%        ft  \
0    0.0  0.378378  0.473684  0.450142  0.28  0.391304  0.318949  0.578947   
1    0.0  0.486486  0.210526  0.857550  0.28  0.239130  0.455910  0.421053   
2    0.0  0.189189  0.350877  0.321937  0.36  0.673913  0.255159  0.684211   
3    0.0  0.297297  0.368421  0.441595  0.32  0.304348  0.454034  0.605263   
4    0.0  0.351351  0.192982  0.692308  0.52  0.304348  0.776735  0.684211   
..   ...       ...       ...       ...   ...       ...       ...       ...   
987  0.0  0.270270  0.280702  0.487179  0.56  0.434783  0.664165  0.763158   
988  0.0  0.351351  0.333333  0.541311  0.52  0.521739  0.529081  0.526316   
989  0.0  0.297297  0.245614  0.558405  0.56  0.369565  0.744841  0.552632   
990  0.0  0.108108  0.438596  0.159544  0.28  0.608696  0.198874  0.421053   
991  0.0  0.297297  0.508772  0.327635  0.48  0.673913  0.377111  0.552632   

          fta       ft%  ...  usg%_max_opp_10_y  ortg_max_opp_1